<a href="https://colab.research.google.com/github/newton143/Econ8310/blob/master/Assignment3_LSTMClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
import gzip
import numpy as np
import struct
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [4]:
# Custom Dataset using requests
class FashionMNIST(Dataset):
    def __init__(self, train=True):
        self.train = train

        # URLs for downloading dataset
        self.urls = {
            "train_images": "https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/train-images-idx3-ubyte.gz",
            "train_labels": "https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/train-labels-idx1-ubyte.gz",
            "test_images": "https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/t10k-images-idx3-ubyte.gz",
            "test_labels": "https://github.com/zalandoresearch/fashion-mnist/raw/b2617bb6d3ffa2e429640350f613e3291e10b141/data/fashion/t10k-labels-idx1-ubyte.gz"
        }

        self.load_data()

    def download_and_extract(self, url, header_size):
        print(f"Downloading from: {url}")
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Just in case
        with gzip.GzipFile(fileobj=response.raw) as f:
            return f.read()[header_size:]

    def load_data(self):
      if self.train:
          image_data = self.download_and_extract(self.urls["train_images"], 16)
          label_data = self.download_and_extract(self.urls["train_labels"], 8)
          self.images = np.frombuffer(image_data, dtype=np.uint8).reshape(-1, 28, 28)
          self.labels = np.frombuffer(label_data, dtype=np.uint8)
      else:
          image_data = self.download_and_extract(self.urls["test_images"], 16)
          label_data = self.download_and_extract(self.urls["test_labels"], 8)
          self.images = np.frombuffer(image_data, dtype=np.uint8).reshape(-1, 28, 28)
          self.labels = np.frombuffer(label_data, dtype=np.uint8)

      print(f"{'Train' if self.train else 'Test'} samples loaded: {len(self.labels)}")


    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = torch.tensor(self.images[idx], dtype=torch.float32) / 255.0  # Normalize
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label


In [5]:
# DataLoader setup
train_dataset = FashionMNIST(train=True)
test_dataset = FashionMNIST(train=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

Train samples loaded: 60000
Test samples loaded: 10000


In [6]:
# LSTM Model
class LSTMFashionClassifier(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_layers=2, num_classes=10):
        super(LSTMFashionClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))  # x: [batch, 28, 28]
        out = self.fc(out[:, -1, :])     # Get last time step
        return out



In [9]:
# Instantiate model, loss, optimizer
model = LSTMFashionClassifier()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
# Training function
def train_epoch(dataloader, model, loss_fn, optimizer):
    model.train()
    for batch_idx, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = loss_fn(pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 10 == 0:
            print(f"Train Loss: {loss.item():.4f} [{batch_idx * len(X)}/{len(dataloader.dataset)}]")

In [12]:
# Evaluation function
def evaluate(dataloader, model, loss_fn):
    model.eval()
    total_loss = 0
    correct = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            total_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).sum().item()

    acc = 100 * correct / len(dataloader.dataset)
    avg_loss = total_loss / len(dataloader)
    print(f"Test Accuracy: {acc:.2f}%, Avg Loss: {avg_loss:.4f}")

In [13]:
# Training loop
epochs = 20
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}")
    train_epoch(train_loader, model, loss_fn, optimizer)
    evaluate(test_loader, model, loss_fn)

print("Training completed")


Epoch 1
Train Loss: 2.3084 [0/60000]
Train Loss: 2.2247 [640/60000]
Train Loss: 1.8732 [1280/60000]
Train Loss: 1.5435 [1920/60000]
Train Loss: 1.4361 [2560/60000]
Train Loss: 0.9837 [3200/60000]
Train Loss: 0.9712 [3840/60000]
Train Loss: 0.9968 [4480/60000]
Train Loss: 0.9474 [5120/60000]
Train Loss: 0.8460 [5760/60000]
Train Loss: 0.5504 [6400/60000]
Train Loss: 0.9593 [7040/60000]
Train Loss: 0.7455 [7680/60000]
Train Loss: 0.6235 [8320/60000]
Train Loss: 0.6058 [8960/60000]
Train Loss: 0.8511 [9600/60000]
Train Loss: 0.6033 [10240/60000]
Train Loss: 0.7066 [10880/60000]
Train Loss: 0.7711 [11520/60000]
Train Loss: 0.8160 [12160/60000]
Train Loss: 0.8215 [12800/60000]
Train Loss: 0.7427 [13440/60000]
Train Loss: 0.8197 [14080/60000]
Train Loss: 0.6975 [14720/60000]
Train Loss: 0.6661 [15360/60000]
Train Loss: 0.7344 [16000/60000]
Train Loss: 0.6848 [16640/60000]
Train Loss: 0.6074 [17280/60000]
Train Loss: 0.7330 [17920/60000]
Train Loss: 0.4642 [18560/60000]
Train Loss: 0.6071 [1